In [1]:
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, MessagesState, START, END
from dotenv import load_dotenv
import os

In [2]:
# Load API key from .env file
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# Initialize the LLM (using OpenAI's GPT-4o)
model = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

# Node function to handle the user query and call the LLM
def call_llm(state: MessagesState):
    messages = state["messages"]
    response = model.invoke(messages[-1].content)
    return {"messages": [response]}

# Define the graph
workflow = StateGraph(MessagesState)

# Add the node to call the LLM
workflow.add_node("call_llm", call_llm)

# Define the edges (start -> LLM -> end)
workflow.add_edge(START, "call_llm")
workflow.add_edge("call_llm", END)

# Compile the workflow
app = workflow.compile()

In [3]:
# Function to continuously take user input
def interact_with_agent():
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            print("Ending the conversation.")
            break
        input_message = {
            "messages": [("human", user_input)]
        }

        for chunk in app.stream(input_message, stream_mode="values"):
            chunk["messages"][-1].pretty_print()

# Start interacting with the agent
interact_with_agent()

================================ Human Message =================================

Who is narendra modi
================================== Ai Message ==================================

Narendra Modi is an Indian politician who has been serving as the Prime Minister of India since May 2014. He is a member of the Bharatiya Janata Party (BJP) and the Rashtriya Swayamsevak Sangh (RSS), a Hindu nationalist organization. Born on September 17, 1950, in Vadnagar, Gujarat, Modi came from a modest background and worked in various capacities before entering politics.

He served as the Chief Minister of Gujarat from 2001 to 2014, where he was credited with significant economic growth in the state but also faced criticism regarding his government's handling of the 2002 Gujarat riots. As Prime Minister, Modi has focused on various initiatives including economic reform, digitalization, skill development, and infrastructure development. He is known for his strong leadership style and has been a contro